#Installs

In [1]:
# Basic stuff
import numpy as np
import pandas as pd
# importing libraries for data visualisations
from matplotlib import pyplot
import matplotlib.pyplot as plt
# Feature engineering
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from imblearn.over_sampling import SMOTE
#Evaluation and saving models
import joblib
# Importing Required Libraries
import pandas as pd
import numpy as np 
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from sklearn.metrics import accuracy_score, mean_squared_error
from statistics import mean
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

#Loading and initial data clean up

In [4]:
#importing data into variables
employee_survey = pd.read_csv('https://raw.githubusercontent.com/CNielsen94/Exercises_AAUBSDS/main/NN_assignments/Assignment%201%20-%20Simple%20NN/employee_survey_data.csv')
df_general = pd.read_csv('https://raw.githubusercontent.com/CNielsen94/Exercises_AAUBSDS/main/NN_assignments/Assignment%201%20-%20Simple%20NN/general_data.csv')
manager_survey = pd.read_csv('https://raw.githubusercontent.com/CNielsen94/Exercises_AAUBSDS/main/NN_assignments/Assignment%201%20-%20Simple%20NN/manager_survey_data.csv')

In [5]:
employee_survey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               4410 non-null   int64  
 1   EnvironmentSatisfaction  4385 non-null   float64
 2   JobSatisfaction          4390 non-null   float64
 3   WorkLifeBalance          4372 non-null   float64
dtypes: float64(3), int64(1)
memory usage: 137.9 KB


In [6]:
manager_survey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   EmployeeID         4410 non-null   int64
 1   JobInvolvement     4410 non-null   int64
 2   PerformanceRating  4410 non-null   int64
dtypes: int64(3)
memory usage: 103.5 KB


In [7]:
df_general.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4410 non-null   int64  
 1   Attrition                4410 non-null   object 
 2   BusinessTravel           4410 non-null   object 
 3   Department               4410 non-null   object 
 4   DistanceFromHome         4410 non-null   int64  
 5   Education                4410 non-null   int64  
 6   EducationField           4410 non-null   object 
 7   EmployeeCount            4410 non-null   int64  
 8   EmployeeID               4410 non-null   int64  
 9   Gender                   4410 non-null   object 
 10  JobLevel                 4410 non-null   int64  
 11  JobRole                  4410 non-null   object 
 12  MaritalStatus            4410 non-null   object 
 13  MonthlyIncome            4410 non-null   int64  
 14  NumCompaniesWorked      

In [8]:
#Cleaning up some missing values by imputing the mean into employee_survey
employee_survey = employee_survey.fillna(employee_survey.mean())

manager_survey['total_mn'] = (manager_survey['JobInvolvement']+manager_survey['PerformanceRating'])/2
employee_survey['total_em'] = (employee_survey['EnvironmentSatisfaction']+employee_survey['JobSatisfaction']+employee_survey['WorkLifeBalance'])/3
general_df = df_general.dropna(axis=0) # Making it a variable here just in case something breaks before this point (You never know)

In [9]:
#Let's make sure we don't have misaligned dataframes before loading it up

# create a list of all EmployeeIDs that exist in the other dataframe
existing_ids = list(general_df['EmployeeID'].unique())

# drop all rows with EmployeeID not in existing_ids from manager_survey
manager_survey = manager_survey[manager_survey['EmployeeID'].isin(existing_ids)]

# drop all rows with EmployeeID not in existing_ids from employee_survey
employee_survey = employee_survey[employee_survey['EmployeeID'].isin(existing_ids)]

#Database setup + data engineering

In [10]:
import sqlite3

In [11]:
conn = sqlite3.connect('HR_DB.db')
general_df['Attrition_Class'] = ''

<ipython-input-11-8919ec8d48d1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_df['Attrition_Class'] = ''


In [12]:
general_df.to_sql('general', conn, if_exists='append', index=False)

4382

In [13]:
employee_survey.to_sql('employee_survey', conn, if_exists='append',index=False)

4382

In [14]:
manager_survey.to_sql('manager_survey', conn, if_exists='append',index=False)

4382

In [15]:
query = 'SELECT EmployeeID, Age, DistanceFromHome, YearsAtCompany, TotalWorkingYears, MonthlyIncome FROM general'
df_tiers = pd.read_sql(query, conn)

In [16]:
df_tiers.head(5)

,EmployeeID,Age,DistanceFromHome,YearsAtCompany,TotalWorkingYears,MonthlyIncome
0,1,51,6,1,1.0,131160
1,2,31,10,5,6.0,41890
2,3,32,17,5,5.0,193280
3,4,38,2,8,13.0,83210
4,5,32,10,6,9.0,23420


In [17]:
# create a list of our conditions
conditions = [
    (df_tiers['Age'] <= 20),
    (df_tiers['Age'] > 20) & (df_tiers['Age'] <= 30),
    (df_tiers['Age'] > 30) & (df_tiers['Age'] <= 40),
    (df_tiers['Age'] > 40) & (df_tiers['Age'] <= 50),
    (df_tiers['Age'] > 50) & (df_tiers['Age'] <= 60)
    ]

# create a list of the values we want to assign for each condition
values = ['less_20', '20_30', '30_40', '40_50', '50_60']

# create a new column and use np.select to assign values to it using our lists as arguments
df_tiers['Age_tier'] = np.select(conditions, values)

In [18]:
# create a list of our conditions
conditions = [
    (df_tiers['DistanceFromHome'] > 0) & (df_tiers['DistanceFromHome'] <= 5),
    (df_tiers['DistanceFromHome'] > 5) & (df_tiers['DistanceFromHome'] <= 10),
    (df_tiers['DistanceFromHome'] > 10) & (df_tiers['DistanceFromHome'] <= 20),
    (df_tiers['DistanceFromHome'] > 20) & (df_tiers['DistanceFromHome'] <= 29)
    ]

# create a list of the values we want to assign for each condition
values = [4,3,2,1]

# create a new column and use np.select to assign values to it using our lists as arguments
df_tiers['distancehome_tier'] = np.select(conditions, values)

In [19]:
df_general['JobRole'].unique()

array(['Healthcare Representative', 'Research Scientist',
       'Sales Executive', 'Human Resources', 'Research Director',
       'Laboratory Technician', 'Manufacturing Director',
       'Sales Representative', 'Manager'], dtype=object)

In [20]:
# create a list of our conditions
conditions = [
    (df_tiers['YearsAtCompany'] >= 0) & (df_tiers['YearsAtCompany'] <2),
    (df_tiers['YearsAtCompany'] >= 2) & (df_tiers['YearsAtCompany'] <5),
    (df_tiers['YearsAtCompany'] >= 5) & (df_tiers['YearsAtCompany'] <8),
    (df_tiers['YearsAtCompany'] >= 8) & (df_tiers['YearsAtCompany'] <12),
    (df_tiers['YearsAtCompany'] >=12)
    ]

# create a list of the values we want to assign for each condition
values = [1,2,3,4,5]

# create a new column and use np.select to assign values to it using our lists as arguments
df_tiers['YearsAtCompany_tier'] = np.select(conditions, values)

In [21]:
# create a list of our conditions
conditions = [
    (df_tiers['TotalWorkingYears'] >= 0) & (df_tiers['TotalWorkingYears'] <2),
    (df_tiers['TotalWorkingYears'] >= 2) & (df_tiers['TotalWorkingYears'] <5),
    (df_tiers['TotalWorkingYears'] >= 5) & (df_tiers['TotalWorkingYears'] <8),
    (df_tiers['TotalWorkingYears'] >= 8) & (df_tiers['TotalWorkingYears'] <12),
    (df_tiers['TotalWorkingYears'] >=12)
    ]
# create a list of the values we want to assign for each condition
values = [1,2,3,4,5]

# create a new column and use np.select to assign values to it using our lists as arguments
df_tiers['TotalWorkingYear_tier'] = np.select(conditions, values)

In [22]:
# create a list of our conditions
conditions = [
    (df_tiers['MonthlyIncome'] >= 10000) & (df_tiers['MonthlyIncome'] <25000),
    (df_tiers['MonthlyIncome'] >= 25000) & (df_tiers['MonthlyIncome'] <35000),
    (df_tiers['MonthlyIncome'] >= 35000) & (df_tiers['MonthlyIncome'] <60000),
    (df_tiers['MonthlyIncome'] >=60000) & (df_tiers['MonthlyIncome'] <100000),
    (df_tiers['MonthlyIncome'] >=100000)
    ]

# create a list of the values we want to assign for each condition
values = [1,2,3,4,5]

# create a new column and use np.select to assign values to it using our lists as arguments
df_tiers['MonthlyIncome_tier'] = np.select(conditions, values)

In [23]:
df_tiers.head(5)

,EmployeeID,Age,DistanceFromHome,YearsAtCompany,TotalWorkingYears,MonthlyIncome,Age_tier,distancehome_tier,YearsAtCompany_tier,TotalWorkingYear_tier,MonthlyIncome_tier
0,1,51,6,1,1.0,131160,50_60,3,1,1,5
1,2,31,10,5,6.0,41890,30_40,3,3,3,3
2,3,32,17,5,5.0,193280,30_40,2,3,3,5
3,4,38,2,8,13.0,83210,30_40,4,4,5,4
4,5,32,10,6,9.0,23420,30_40,3,3,4,1


In [24]:
new_df = df_tiers[['EmployeeID', 'Age_tier', 'distancehome_tier','YearsAtCompany_tier','TotalWorkingYear_tier','MonthlyIncome_tier']]

In [25]:
new_df.head(5)

,EmployeeID,Age_tier,distancehome_tier,YearsAtCompany_tier,TotalWorkingYear_tier,MonthlyIncome_tier
0,1,50_60,3,1,1,5
1,2,30_40,3,3,3,3
2,3,30_40,2,3,3,5
3,4,30_40,4,4,5,4
4,5,30_40,3,3,4,1


In [26]:
new_df.to_sql('EmployeeTiers', conn, if_exists='append',index=False)

4382

In [27]:
query = 'SELECT * FROM general'
df_general = pd.read_sql(query, conn)

In [28]:
df_general.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EmployeeCount', 'EmployeeID', 'Gender',
       'JobLevel', 'JobRole', 'MaritalStatus', 'MonthlyIncome',
       'NumCompaniesWorked', 'Over18', 'PercentSalaryHike', 'StandardHours',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
       'Attrition_Class'],
      dtype='object')

In [29]:
query = 'SELECT * FROM EmployeeTiers'
employee_tiers = pd.read_sql(query, conn)

In [30]:
employee_tiers.columns

Index(['EmployeeID', 'Age_tier', 'distancehome_tier', 'YearsAtCompany_tier',
       'TotalWorkingYear_tier', 'MonthlyIncome_tier'],
      dtype='object')

In [31]:
query = 'SELECT * FROM employee_survey'
employee_survey = pd.read_sql(query, conn)

In [32]:
employee_survey.columns

Index(['EmployeeID', 'EnvironmentSatisfaction', 'JobSatisfaction',
       'WorkLifeBalance', 'total_em'],
      dtype='object')

In [33]:
query = 'SELECT * FROM manager_survey'
manager_survey = pd.read_sql(query, conn)

In [34]:
manager_survey.columns

Index(['EmployeeID', 'JobInvolvement', 'PerformanceRating', 'total_mn'], dtype='object')